In [30]:
import requests
import json
from termcolor import colored

# Server configuration
SERVER_HOST = '0.0.0.0'
SERVER_PORT = 8001
SERVER_URL = f'http://{SERVER_HOST}:{SERVER_PORT}'

def send_mcp_message(method: str, params: dict = None, message_id: int = 1):
    """
    Send an MCP message to the server and display the request/response.
    """
    # Construct MCP message
    message = {
        "jsonrpc": "2.0",
        "id": message_id,
        "method": method
    }
    
    if params:
        message["params"] = params
    
    print(colored(f"\n📤 Sending MCP Request:", "cyan", attrs=["bold"]))
    print(colored(json.dumps(message, indent=2), "cyan"))
    
    try:
        # Send request to MCP endpoint
        response = requests.post(
            f'{SERVER_URL}/mcp',
            json=message,
            headers={'Content-Type': 'application/json'},
            stream=True,
            timeout=10
        )
        
        # Parse SSE response
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8')
                if line_str.startswith('data: '):
                    data = json.loads(line_str[6:])  # Remove 'data: ' prefix
                    
                    print(colored(f"\n📥 Received MCP Response:", "green", attrs=["bold"]))
                    print(colored(json.dumps(data, indent=2), "green"))
                    
                    return data
        
    except Exception as e:
        print(colored(f"\n❌ Error: {e}", "red"))
        return None

# Test 1: Initialize
print(colored("="*60, "magenta"))
print(colored("TEST 1: MCP INITIALIZATION", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

init_response = send_mcp_message(
    method="initialize",
    params={
        "protocolVersion": "2024-11-05",
        "capabilities": {},
        "clientInfo": {
            "name": "colab-test-client",
            "version": "1.0.0"
        }
    },
    message_id=1
)

if init_response and 'result' in init_response:
    print(colored("\n✅ Initialization successful!", "green", attrs=["bold"]))
    print(f"   Protocol Version: {init_response['result']['protocolVersion']}")
    print(f"   Server: {init_response['result']['serverInfo']['name']}")
else:
    print(colored("\n❌ Initialization failed", "red", attrs=["bold"]))

TEST 1: MCP INITIALIZATION

📤 Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {},
    "clientInfo": {
      "name": "colab-test-client",
      "version": "1.0.0"
    }
  }
}

❌ Error: HTTPConnectionPool(host='0.0.0.0', port=8001): Max retries exceeded with url: /mcp (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1097f2900>: Failed to establish a new connection: [Errno 61] Connection refused'))

❌ Initialization failed


In [7]:
print(colored("="*60, "magenta"))
print(colored("TEST 2: LIST AVAILABLE TOOLS", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

tools_response = send_mcp_message(
    method="tools/list",
    message_id=2
)

if tools_response and 'result' in tools_response:
    tools = tools_response['result']['tools']
    print(colored(f"\n✅ Found {len(tools)} tools:", "green", attrs=["bold"]))
    for i, tool in enumerate(tools, 1):
        print(colored(f"\n{i}. {tool['name']}", "yellow", attrs=["bold"]))
        print(f"   {tool['description']}")
else:
    print(colored("\n❌ Failed to list tools", "red", attrs=["bold"]))

TEST 2: LIST AVAILABLE TOOLS

📤 Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/list"
}

📥 Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "tools": [
      {
        "name": "web_search_tool",
        "description": "Query the local Amazon 2020 vector database for product information from the private catalog. Returns a list of matching items with `sku`, `title`, `price`, `rating`, and, when available, `brand`, `ingredients`, and `doc_id`. Use this tool when the user\u2019s request should be answered from the internal Amazon 2020 slice rather than the public web.",
        "inputSchema": {
          "type": "object",
          "title": "WebSearchInput",
          "properties": {
            "query": {
              "type": "string",
              "description": "Natural language product query to send to the web search API."
            },
            "max_results": {
              "type": "integer",
              "description": "Maximum

In [4]:
print(colored("="*60, "magenta"))
print(colored("TEST 3: LIST ALL CUSTOMERS", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

response = send_mcp_message(
    method="tools/call",
    params={
        "name": "web_search_tool",
        "arguments": {'args': {
            'query': 'Latest advancements in AI technology 2024',
            'num_results': 3
        }}
    },
    message_id=3
)

NameError: name 'colored' is not defined

In [2]:
from google.adk.tools.mcp_tool import McpToolset, StreamableHTTPConnectionParams

mcp = McpToolset(
    connection_params=StreamableHTTPConnectionParams(
        url="http://localhost:8001/mcp"
    )
)

def mcp_call(tool_name: str, arguments: dict):
    """Generic wrapper so LangGraph can call MCP tools."""
    result = mcp.call_tool(tool_name, arguments)
    return result["content"]

In [17]:
from langchain_mcp_adapters.client import MultiServerMCPClient

SERVER_HOST = '0.0.0.0'
SERVER_PORT = 8001
SERVER_URL = f'http://{SERVER_HOST}:{SERVER_PORT}'
MCP_SERVERS = {
        "my_server": {
            "url": SERVER_URL+"/mcp",
            "transport": "http",
        },
    }

client = MultiServerMCPClient(MCP_SERVERS)

tools = await client.get_tools() 

In [22]:
from langchain.agents.factory import create_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

agent = create_agent(
        model=llm, 
        tools=tools
        )

response = await agent.ainvoke({"messages": [{"role": "user", "content": "call web_search_tool with query 'Latest advancements in AI technology 2024' and num_results 3"}]})
response

{'messages': [HumanMessage(content="call web_search_tool with query 'Latest advancements in AI technology 2024' and num_results 3", additional_kwargs={}, response_metadata={}, id='45bd0ce6-ce7a-4be2-85e4-12c71749e2ac'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 363, 'total_tokens': 394, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-ClT2hynzJWEnUr6zuuRdYykUaqhEI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b0bc6-189e-7e21-85cf-eef6eee4b6f2-0', tool_calls=[{'name': 'web_search_tool', 'args': {'query': 'Latest advancements in AI technology 2024', 'max_results': 3}, 'id': 'call_X5yqHtR8ub0RRC

In [27]:
response['messages'][-1].content

'Here are some resources on the latest advancements in AI technology for 2024:\n\n1. [The Use of AI Technology in US 2024 General Election](https://www.google.com/search?ibp=oshop&q=Latest advancements in AI technology 2024&prds=localAnnotatedOfferId:1,catalogid:5500036762021401321,pvo:2,pvt:hg,rds:PC_6027158597472037202%7CPROD_PC_6027158597472037202&gl=us&udm=28&pvorigin=2) - $51.11\n\n2. [Advancements in Generative Artificial Intelligence: Proceedings of 4th ETBS 2024](https://www.google.com/search?ibp=oshop&q=Latest advancements in AI technology 2024&prds=localAnnotatedOfferId:1,catalogid:13244946710926941035,pvo:2,pvt:hg,rds:PC_6027158597472037202%7CPROD_PC_6027158597472037202&gl=us&udm=28&pvorigin=2) - $299.99\n\n3. [AI 2024: Advances in Artificial Intelligence](https://www.google.com/search?ibp=oshop&q=Latest advancements in AI technology 2024&prds=localAnnotatedOfferId:1,catalogid:4622644780503239739,pvo:2,pvt:hg,rds:PC_6027158597472037202%7CPROD_PC_6027158597472037202&gl=us&udm

In [28]:
response['messages']

[HumanMessage(content="call web_search_tool with query 'Latest advancements in AI technology 2024' and num_results 3", additional_kwargs={}, response_metadata={}, id='45bd0ce6-ce7a-4be2-85e4-12c71749e2ac'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 363, 'total_tokens': 394, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-ClT2hynzJWEnUr6zuuRdYykUaqhEI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b0bc6-189e-7e21-85cf-eef6eee4b6f2-0', tool_calls=[{'name': 'web_search_tool', 'args': {'query': 'Latest advancements in AI technology 2024', 'max_results': 3}, 'id': 'call_X5yqHtR8ub0RRCo2DeGaYGN5', '

In [29]:
# Extract the final tool message from the response
tool_message = response['messages'][-2].content
print(colored("Final Tool Message:", "yellow", attrs=["bold"]))
print(json.dumps(tool_message, indent=2))

Final Tool Message:
[
  {
    "type": "text",
    "text": "{\"results\": [{\"title\": \"The Use of AI Technology in US 2024 General Election\", \"url\": \"https://www.google.com/search?ibp=oshop&q=Latest advancements in AI technology 2024&prds=localAnnotatedOfferId:1,catalogid:5500036762021401321,pvo:2,pvt:hg,rds:PC_6027158597472037202%7CPROD_PC_6027158597472037202&gl=us&udm=28&pvorigin=2\", \"snippet\": null, \"price\": \"$51.11\", \"availability\": null, \"rating\": null, \"rating_count\": null}, {\"title\": \"Advancements in Generative Artificial Intelligence: Proceedings of 4th ETBS 2024\", \"url\": \"https://www.google.com/search?ibp=oshop&q=Latest advancements in AI technology 2024&prds=localAnnotatedOfferId:1,catalogid:13244946710926941035,pvo:2,pvt:hg,rds:PC_6027158597472037202%7CPROD_PC_6027158597472037202&gl=us&udm=28&pvorigin=2\", \"snippet\": null, \"price\": \"$299.99\", \"availability\": null, \"rating\": null, \"rating_count\": null}, {\"title\": \"AI 2024: Advances in A